In [ ]:
!nvidia-smi

Sun Jan 19 12:06:48 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Install packages

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install mmcv==2.0.1 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html

!pip install -qq ftfy openmim future tensorboard scikit-learn pandas
!mim install mmengine
!git clone -b main https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 520.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89991 sha256=6de09ef656fe89a7b77145a6751d5add5ca376f3fac11e677d381e4ec4a31542
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled to

In [ ]:
# Ensure that you run the notebook from the mmdetection directory
%cd /workspace/mmdetection

[Errno 2] No such file or directory: '/workspace/mmdetection'
/content/mmdetection


# 2. Setup the dataset

In [ ]:
# Download dataset from Google Drive
!gdown 16T40TdpaB8VXohm50SySREwrzbuPcJBC
!mkdir -p data/images
!unzip -qqnd data/images images.zip
!mv Train.csv Test.csv SampleSubmission.csv data/

Downloading...
From (original): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC
From (redirected): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC&confirm=t&uuid=f4bbdb37-2a1a-4b48-b93c-f23ccb26944d
To: /content/mmdetection/images.zip
100% 4.29G/4.29G [00:52<00:00, 82.1MB/s]
mv: cannot stat 'Train.csv': No such file or directory
mv: cannot stat 'Test.csv': No such file or directory
mv: cannot stat 'SampleSubmission.csv': No such file or directory


In [ ]:
!ls data

images


In [ ]:
## Download pretrained model weights
!mkdir -p checkpoints
!cd checkpoints && wget -nc https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth


--2025-01-19 12:12:23--  https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 163.181.42.245, 163.181.42.239, 163.181.42.204, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|163.181.42.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 883034647 (842M) [application/octet-stream]
Saving to: ‘ddq_detr_swinl_30e.pth’

ddq_detr_swinl_30e. 100%[===================>] 842.13M  28.5MB/s    in 27s     

2025-01-19 12:12:50 (31.0 MB/s) - ‘ddq_detr_swinl_30e.pth’ saved [883034647/883034647]



# 3. Training

In [ ]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings,logging

from tqdm import tqdm
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,StratifiedGroupKFold
import sklearn.metrics as skm
from sklearn import preprocessing
import torch
from torch import nn
import torch.nn.functional as F
import cv2

from mmengine import Config
from mmengine.runner import Runner
import mmdet

N_SPLITS = 5
RANDOM_STATE = 41
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Upload the dataset into data

In [ ]:
DIR_DATA = 'data'
CLASSES = {'Trophozoite':0,'WBC':1}
df = pd.read_csv(f'{DIR_DATA}/Train.csv')
df['path'] = f'{DIR_DATA}/images/'+df.Image_ID
df['category_id'] = df['class'].map(CLASSES).fillna(-1)
df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,path,category_id
0,id_u3q6jdck4j.jpg,Trophozoite,1.0,712,1241,737,1270,data/images/id_u3q6jdck4j.jpg,0.0
1,id_a6cl90trri.jpg,Trophozoite,1.0,558,1566,600,1604,data/images/id_a6cl90trri.jpg,0.0
2,id_qvc2le9sm8.jpg,Trophozoite,1.0,1317,2788,1448,2914,data/images/id_qvc2le9sm8.jpg,0.0
3,id_w8xnbd5rvm.jpg,Trophozoite,1.0,925,1744,1041,1823,data/images/id_w8xnbd5rvm.jpg,0.0
4,id_6dop09rk02.jpg,NEG,1.0,0,0,0,0,data/images/id_6dop09rk02.jpg,-1.0
...,...,...,...,...,...,...,...,...,...
23525,id_vxfv34g53d.jpg,WBC,1.0,2336,2441,2537,2647,data/images/id_vxfv34g53d.jpg,1.0
23526,id_61axoyt1e6.jpg,WBC,1.0,486,713,543,770,data/images/id_61axoyt1e6.jpg,1.0
23527,id_ipmba1040m.jpg,Trophozoite,1.0,456,497,486,535,data/images/id_ipmba1040m.jpg,0.0
23528,id_r2x9t2l5cd.jpg,Trophozoite,1.0,2974,1039,3094,1165,data/images/id_r2x9t2l5cd.jpg,0.0


In [ ]:
df.describe()

,confidence,ymin,xmin,ymax,xmax,category_id
count,23530.0,23530.000000,23530.000000,23530.000000,23530.000000,23530.000000
mean,1.0,802.795028,1292.343731,878.255249,1366.806290,0.268423
std,0.0,676.145932,860.772798,704.865276,886.518521,0.504838
min,1.0,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,1.0,317.000000,685.000000,373.000000,740.000000,0.000000
50%,1.0,619.000000,1107.000000,681.000000,1169.000000,0.000000
75%,1.0,1016.750000,1621.000000,1108.000000,1701.000000,1.000000
max,1.0,3012.000000,4051.000000,3116.000000,4155.000000,1.000000


In [ ]:
df.Image_ID.value_counts()

,count
Image_ID,
id_mzq1jmom69.jpg,92
id_tmtzoe9bns.jpg,85
id_7tgo99b7gy.jpg,74
id_1xpbtfb4xh.jpg,74
id_n5talfnmmk.jpg,73
...,...
id_calvvsgfft.jpg,1
id_teng746pbq.jpg,1
id_0nkc2ttx1k.jpg,1


In [ ]:
# df['area'] = (df.xmax-df.xmin)*(df.ymax-df.ymin)
# df.area.describe()

In [ ]:
df['class'].value_counts()

,count
class,
Trophozoite,15838
WBC,7004
NEG,688


In [ ]:
def split_data(df):
    gkf  = StratifiedGroupKFold(n_splits=N_SPLITS,shuffle=True,random_state=RANDOM_STATE)
    df['fold'] = -1
    for fold_id, (train_index, test_index) in enumerate(gkf.split(df,y=df['class'],groups=df.Image_ID)):
        df.loc[test_index,'fold'] = fold_id
    return df

def get_splits(df,fold):
    df_trn = df[df.fold!=fold].copy()
    df_val = df[df.fold==fold].copy()

    return df_trn,df_val

df = split_data(df)
df

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,path,category_id,fold
0,id_u3q6jdck4j.jpg,Trophozoite,1.0,712,1241,737,1270,data/images/id_u3q6jdck4j.jpg,0.0,0
1,id_a6cl90trri.jpg,Trophozoite,1.0,558,1566,600,1604,data/images/id_a6cl90trri.jpg,0.0,1
2,id_qvc2le9sm8.jpg,Trophozoite,1.0,1317,2788,1448,2914,data/images/id_qvc2le9sm8.jpg,0.0,0
3,id_w8xnbd5rvm.jpg,Trophozoite,1.0,925,1744,1041,1823,data/images/id_w8xnbd5rvm.jpg,0.0,0
4,id_6dop09rk02.jpg,NEG,1.0,0,0,0,0,data/images/id_6dop09rk02.jpg,-1.0,0
...,...,...,...,...,...,...,...,...,...,...
23525,id_vxfv34g53d.jpg,WBC,1.0,2336,2441,2537,2647,data/images/id_vxfv34g53d.jpg,1.0,4
23526,id_61axoyt1e6.jpg,WBC,1.0,486,713,543,770,data/images/id_61axoyt1e6.jpg,1.0,4
23527,id_ipmba1040m.jpg,Trophozoite,1.0,456,497,486,535,data/images/id_ipmba1040m.jpg,0.0,2
23528,id_r2x9t2l5cd.jpg,Trophozoite,1.0,2974,1039,3094,1165,data/images/id_r2x9t2l5cd.jpg,0.0,4


In [ ]:
df['split'] = 'train'
df.loc[df.fold==FOLD,'split'] = 'val'
df.split.value_counts()

,count
split,
train,18791
val,4739


In [ ]:
def gen_anno(meta,split):
    annotations = []
    images = []
    anno_id = 0
    image_id = 0

    meta=meta.copy()
    for file_name,d in tqdm(meta.groupby('Image_ID')):
        path = d.path.values[0]
        im = cv2.imread(path)
        height,width,_ = im.shape
        for _,row in d.iterrows():
            # print(file_name,width,row.width,height,row.height)
            # assert width==row.width
            # assert height==row.height
            x0, y0, x1, y1 = row.xmin,row.ymin,row.xmax,row.ymax
            w = x1 - x0
            h = y1 - y0
            bbox = np.array([x0, y0, w, h]).tolist()
            area = w * h
            if area<1:
                #print('skipping',file_name,area)
                continue

            anno = dict(
                        image_id = image_id,
                        id = anno_id,
                        category_id = row.category_id,
                        bbox = bbox,
                        area = area,
                        iscrowd = 0
                    )
            anno_id += 1
            annotations.append(anno)

        images.append(dict(id=image_id, file_name=file_name,height=height,width=width))
        image_id += 1


    coco_json = dict(images=images, annotations=annotations,
                        categories=[dict(id=0,name='TRO'),dict(id=1,name='WBC')])
    path = f'data/{split}.json'
    print(f'written {len(annotations)} annotations for {len(images)} images to {path}')
    with open(path,'w', encoding='utf-8') as f:
        json.dump(coco_json,f,ensure_ascii=False)

In [ ]:
gen_anno(df[df.split=='val'],'val')
### gen_anno(df[df.split=='train'],'train')
# training on complete dataset
gen_anno(df,'train')

100%|██████████| 553/553 [00:39<00:00, 13.99it/s]


written 4603 annotations for 553 images to data/val.json


100%|██████████| 2747/2747 [03:11<00:00, 14.34it/s]


written 22841 annotations for 2747 images to data/train.json


In [ ]:
!ls data

images	Test.csv  Train.csv  train.json  val.json


In [ ]:
cfg_path = 'configs/ddq/ddq-detr-4scale_swinl_8xb2-30e_coco.py'
cfg = Config.fromfile(cfg_path)

load_from = 'checkpoints/ddq_detr_swinl_30e.pth'

data_dir = 'data'
img_prefix = 'images'
max_epochs = 5
val_interval = 1

bs = 1
cfg.train_num_workers = num_workers = 8
cfg.train_dataloader.batch_size = bs

cfg.data_root = data_dir
cfg.work_dir = './output'
metainfo = {
    'classes': ('TRO','WBC'),
    'palette': [
        (128, 60, 30),
        (220, 20, 60),
    ]
}
cfg.dataset_type = 'CocoDataset'

num_classes = 2

cfg.num_classes = num_classes
cfg.model.bbox_head.num_classes = num_classes

cfg.train_pipeline =[
    dict(backend_args=None, type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(prob=0.5, type='RandomFlip'),

    dict(
        transforms=[
            [
                dict(
                    keep_ratio=True,
                    scales=[
                        (
                            480,
                            1333,
                        ),
                        (
                            512,
                            1333,
                        ),
                        (
                            544,
                            1333,
                        ),
                        (
                            576,
                            1333,
                        ),
                        (
                            608,
                            1333,
                        ),
                        (
                            640,
                            1333,
                        ),
                        (
                            672,
                            1333,
                        ),
                        (
                            704,
                            1333,
                        ),
                        (
                            736,
                            1333,
                        ),
                        (
                            768,
                            1333,
                        ),
                        (
                            800,
                            1333,
                        ),
                    ],
                    type='RandomChoiceResize'),
            ],
            [
                dict(
                    keep_ratio=True,
                    scales=[
                        (
                            400,
                            1200,
                        ),
                        (
                            500,
                            1200,
                        ),
                        (
                            600,
                            1200,
                        ),
                    ],
                    type='RandomChoiceResize'),
                dict(
                    allow_negative_crop=True,
                    crop_size=(
                        384,
                        600,
                    ),
                    crop_type='absolute_range',
                    type='RandomCrop'),
                dict(
                    keep_ratio=True,
                    scales=[
                        (
                            480,
                            1333,
                        ),
                        (
                            512,
                            1333,
                        ),
                        (
                            544,
                            1333,
                        ),
                        (
                            576,
                            1333,
                        ),
                        (
                            608,
                            1333,
                        ),
                        (
                            640,
                            1333,
                        ),
                        (
                            672,
                            1333,
                        ),
                        (
                            704,
                            1333,
                        ),
                        (
                            736,
                            1333,
                        ),
                        (
                            768,
                            1333,
                        ),
                        (
                            800,
                            1333,
                        ),
                    ],
                    type='RandomChoiceResize'),
            ],
        ],
        type='RandomChoice'),

    dict(type='YOLOXHSVRandomAug'),
    dict(type='Sharpness',prob=0.5),
    dict(type='AutoContrast',prob=0.5,min_mag=0.1,max_mag=1.9,level=10),
    dict(type='Rotate', level=10, min_mag=180.,max_mag=180.,prob=0.5),

    dict(type='PackDetInputs'),
    ]


cfg.train_dataloader.dataset=dict(
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='train.json',
        backend_args=None,
        data_prefix=dict(img=img_prefix),
        filter_cfg=dict(filter_empty_gt=False),
        pipeline=cfg.train_pipeline,

        type='CocoDataset')

cfg.train_dataloader.num_workers = cfg.val_dataloader.num_workers = num_workers
cfg.val_dataloader.dataset=dict(
        data_root=cfg.data_root,
        metainfo=metainfo,
        data_prefix=dict(img=img_prefix),
        ann_file='val.json',
        pipeline = cfg.test_pipeline,
        test_mode=True,
    type='CocoDataset')

cfg.test_dataloader = cfg.val_dataloader

cfg.load_from = load_from
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=val_interval)
cfg.visualizer = dict( name='visualizer',type='DetLocalVisualizer',vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])
cfg.val_evaluator = cfg.test_evaluator = dict(
    ann_file='data/val.json',
    backend_args=None,
    format_only=False,
    metric=[
        'bbox',
    ],
    type='CocoMetric')


In [ ]:
fix_seed(RANDOM_STATE)
runner = Runner.from_cfg(cfg)
runner.train()

01/19 13:19:03 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1077708736
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.2, V12.2.140
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm

/usr/local/lib/python3.11/dist-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmdet.visualization.local_visualizer.DetLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 14.75 GiB total capacity; 14.40 GiB already allocated; 19.06 MiB free; 14.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# !cp output/epoch_30.pth ./